# How to use CrewAI Planning in your Crew of agents

---

The planning feature in CrewAI allows you to add planning capability to your crew. When enabled, before each Crew iteration, all Crew information is sent to an AgentPlanner that will plan the tasks step by step, and this plan will be added to each task description.

This guide will serve as a demostration of how to use the **planning** feature in your crews. Let's start by installing required packages.

### Install dependencies

In [ ]:
%pip install -U --quiet crewai crewai-tools exa_py
%pip install --quiet --upgrade ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.0/657.0 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta

In [ ]:
import warnings

# Filter out deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', message='Pydantic serializer warnings')

### Configure API keys for both the LLM and Exa for searching the web.

In [ ]:
import os
from crewai import LLM
from crewai_tools import EXASearchTool

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
EXA_API_KEY = os.getenv('EXA_API_KEY')

openai_model = LLM(model="openai/gpt-4o-mini", api_key=OPENAI_API_KEY)
tool = EXASearchTool(api_key=EXA_API_KEY)

### Now we build our agents!
 
- **Analyst** to do research and a 
- **Reporter** to generate a detailed report from the research.
- In our **crew** we will add **`planning=True`** to enable planning capabilities for our agents before they perform the tasks. 
- Here's where we also add the **`planning_llm`** that will be used to plan the tasks

In [ ]:
from crewai import Agent, Task, Crew

# Create an agent with reasoning enabled
analyst = Agent(
    role="Research Specialist",
    goal="Conduct detailed research on {topic} using the latest information as of {todays_date}.",
    backstory="You are an expert at web research.",
    llm=openai_model,
    tools=[tool],
    verbose=True
)

reporter = Agent(
    role="Report Specialist",
    goal="Generate a detailed report on {topic} using the latest information as of {todays_date}.",
    backstory="You are an expert at executive report writing",
    llm=openai_model,
    verbose=True
)

# Create tasks
analysis_task = Task(
    description="Research and analyze the recent trends and generate a detailed report on {topic}",
    expected_output="A detailed report structured as an executive report for c-suite execs to read.",
    agent=analyst,
    verbose=True
)

report_task = Task(
    description="Generate a detailed report on {topic}",
    expected_output="A detailed report structured as an executive report for c-suite execs to read.",
    agent=analyst,
    markdowns=True,
    verbose=True
)

# Create a crew and run the task
crew = Crew(
    agents=[analyst, reporter],
    tasks=[analysis_task, report_task],
    planning=True,
    planning_llm=openai_model,
    verbose=True
)

result = crew.kickoff(inputs={"topic":"Influencer marketing", "todays_date": "July 31, 2025"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: b2ecb48f-8b8a-4466-8c6b-85f573ae7701                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

 
[2025-07-31 17:15:13][INFO]: Planning the crew execution


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7d7edb77-db63-4b06-804c-673854e4c4f3                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Specialist                                                                                     │
│                                                                                                                 │
│  Task: Research and analyze the recent trends and generate a detailed report on Influencer marketing1. Begin    │
│  by defining the specific focus areas of Influencer Marketing based on current industry needs, such as          │
│  platforms used, audience engagement strategies, or ROI measurement techniques.                                 │
│  2. Use the EXASearchTool to conduct in-depth searches:                                                         │
│     - Input the search query as 'Influencer Marketing Trends 2025'.                                             │
│     - Set the start_publish_date as '2023-01-01' and end_publish_date as '2025-07-31' to filter for recent      │
│  articles.                                                                                                      │
│     - Consider including specific domains known for marketing analytics and reports if necessary.               │
│  3. Analyze the information gathered from various sources, identifying key trends, challenges, and emerging     │
│  practices.                                                                                                     │
│  4. Create a structured outline for the report based on the significant findings from the research, ensuring    │
│  it adheres to the executive report format.                                                                     │
│  5. Summarize findings and insights succinctly, focusing on actionable strategies relevant for C-suite execs.   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Title: Influencer Marketing Trends to Know of in 2025 - Taboola.com                                            │
│  URL: https://www.taboola.com/marketing-hub/influencer-marketing-trends/                                        │
│  ID: https://www.taboola.com/marketing-hub/influencer-marketing-trends/                                         │
│  Score: None                                                                                                    │
│  Published Date: 2025-07-29T17:16:40.681Z                                                                       │
│  Author:                                                                                                        │
│  Image: None                                                                                                    │
│  Favicon: None                                                                                                  │
│  Extras: None                                                                                                   │
│  Subpages: None                                                                                                 │
│  Text: None                                                                                                     │
│  Highlights: None                                                                                               │
│  Highlight Scores: None                                                                                         │
│  Summary: None                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
│  Title: Back-to-School Influencer Marketing Tips for 2025 - Captiv8 Blog                                        │
│  URL: https://captiv8.io/blog/2025/07/29/back-to-school-influencer-marketing-tips-for-2025/                     │
│  ID: https://captiv8.io/blog/2025/07/29/back-to-school-influencer-marketing-tips-for-2025/                      │
│  Score: None                                                                                                    │
│  Published Date: 2025-07-29T17:16:40.681Z                                                                       │
│  Author: None                                                                                                   │
│  Image: None                                                                                                    │
│  Favicon: None                                                                                                  │
│  Extras: None                                                                                                   │
│  Subpages: None                                                                                                 │
│  Text: None                                                                                                     │
│  Highlights: None                                                                                               │
│  Highlight Scores: None                                                                                         │
│  Summary: None                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
│  Title: Fall 2025 Influencer Marketing Trends to Watch

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Specialist                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Executive Report on Influencer Marketing Trends (2025)                                                       │
│                                                                                                                 │
│  ## I. Executive Summary                                                                                        │
│  Influencer marketing continues to evolve rapidly in 2025, marked by:                                           │
│  - Increased reliance on **micro and nano influencers** for authentic engagement.                               │
│  - Integration of **artificial intelligence** for campaign optimization, enabling personalized experiences at   │
│  scale.                                                                                                         │
│  - Enhanced methods for **measuring return on investment (ROI)**, focusing on data-driven insights.             │
│                                                                                                                 │
│  ## II. Current Trends in Influencer Marketing                                                                  │
│  ### A. Shift to Micro and Nano Influencers                                                                     │
│  - **Authenticity and Engagement:** Brands are shifting focus towards influencers with smaller, more engaged    │
│  audiences to foster genuine connections.                                                                       │
│                                                                                                                 │
│  ### B. Rise of AI in Campaign Optimization                                                                     │
│  - **Automation and Data-Driven Strategies:** AI is being leveraged for analyzing consumer behavior,            │
│  optimizing content delivery, and personalizing audience interactions.                                          │
│                                                                                                                 │
│  ### C. Growth of Multi-Channel Strategies                                                                      │
│  - **Utilizing Various Platforms Effectively:** Marketers are embracing multi-channel strategies that include   │
│  TikTok, Instagram, and YouTube, ensuring wider reach and diverse audience engagement.                          │
│                                                                                                                 │
│  ## III. Audience Engagement Strategies                                                                         │
│  ### A. Video Content Proliferation                                                                             │
│  - **Understanding Audience Preferences:** There is a marked preference for video content across platforms,     │
│  emphasizing the importance of impactful storytelling.                                                          │
│                                                                                                                 │
│  ### B. Platform-Specific Features                                                                              │
│  - **Tailoring Approaches:** Strategies are being tailo

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4ad839e0-6e57-4598-9ac3-035335a986d2                                                                     │
│  Agent: Research Specialist                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: b2ecb48f-8b8a-4466-8c6b-85f573ae7701                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Executive Report on Influencer Marketing Trends (2025)                                                       │
│                                                                                                                 │
│  ## I. Executive Summary                                                                                        │
│  Influencer marketing continues to evolve rapidly in 2025, marked by:                                           │
│  - Increased reliance on **micro and nano influencers** for authentic engagement.                               │
│  - Integration of **artificial intelligence** for campaign optimization, enabling personalized experiences at   │
│  scale.                                                                                                         │
│  - Enhanced methods for **measuring return on investment (ROI)**, focusing on data-driven insights.             │
│                                                                                                                 │
│  ## II. Current Trends in Influencer Marketing                                                                  │
│  ### A. Shift to Micro and Nano Influencers                                                                     │
│  - **Authenticity and Engagement:** Brands are shifting focus towards influencers with smaller, more engaged    │
│  audiences to foster genuine connections.                                                                       │
│                                                                                                                 │
│  ### B. Rise of AI in Campaign Optimization                                                                     │
│  - **Automation and Data-Driven Strategies:** AI is being leveraged for analyzing consumer behavior,            │
│  optimizing content delivery, and personalizing audience interactions.                                          │
│                                                                                                                 │
│  ### C. Growth of Multi-Channel Strategies                                                                      │
│  - **Utilizing Various Platforms Effectively:** Marketers are embracing multi-channel strategies that include   │
│  TikTok, Instagram, and YouTube, ensuring wider reach and diverse audience engagement.                          │
│                                                                                                                 │
│  ## III. Audience Engagement Strategies                                                                         │
│  ### A. Video Content Proliferation                                                                             │
│  - **Understanding Audience Preferences:** There is a marked preference for video content across platforms,     │
│  emphasizing the importance of impactful storytelling.                                                          │
│                                                                                                                 │
│  ### B. Platform-Specific Features                    

In [ ]:
from IPython.display import Markdown

Markdown(result.raw)

```markdown
# Executive Report on Influencer Marketing Trends (2025)

## I. Executive Summary
Influencer marketing continues to evolve rapidly in 2025, marked by:
- Increased reliance on **micro and nano influencers** for authentic engagement.
- Integration of **artificial intelligence** for campaign optimization, enabling personalized experiences at scale.
- Enhanced methods for **measuring return on investment (ROI)**, focusing on data-driven insights.

## II. Current Trends in Influencer Marketing
### A. Shift to Micro and Nano Influencers
- **Authenticity and Engagement:** Brands are shifting focus towards influencers with smaller, more engaged audiences to foster genuine connections.

### B. Rise of AI in Campaign Optimization
- **Automation and Data-Driven Strategies:** AI is being leveraged for analyzing consumer behavior, optimizing content delivery, and personalizing audience interactions.

### C. Growth of Multi-Channel Strategies
- **Utilizing Various Platforms Effectively:** Marketers are embracing multi-channel strategies that include TikTok, Instagram, and YouTube, ensuring wider reach and diverse audience engagement.

## III. Audience Engagement Strategies
### A. Video Content Proliferation
- **Understanding Audience Preferences:** There is a marked preference for video content across platforms, emphasizing the importance of impactful storytelling.

### B. Platform-Specific Features
- **Tailoring Approaches:** Strategies are being tailored to specific platform features, for instance:
  - Utilizing **Instagram Reels** for bite-sized storytelling.
  - Engaging audiences on **TikTok** with humorous and relatable content.

## IV. Measuring ROI in Influencer Marketing
### A. Key Performance Indicators (KPIs)
- **Engagement Rates and Conversion Metrics:** Brands are identifying crucial KPIs such as click-through rates, engagement metrics, and conversion rates to measure campaign effectiveness.

### B. Utilizing Benchmark Reports
- **Understanding Market Standards:** Benchmark reports provide essential insights on cost and performance, enabling brands to compare their efforts and strategize accordingly.

## V. Challenges in Influencer Marketing
### A. Navigating Changing Algorithms
- **Adapting to Platform Changes:** Frequent algorithm updates require brands to remain agile and adjust strategies to maintain visibility and engagement.

### B. Managing Authenticity vs. Commercialization
- **Maintaining Audience Trust:** Brands must balance promotional activities with maintaining the authenticity that audiences expect from influencer partnerships.

## VI. Future Directions
- As influencer marketing continues to evolve, the focus will remain on **authenticity**, **diversification**, and **results-driven methodologies**.
- Strategic recommendations for C-suite executives include investing in data analytics to enhance campaign effectiveness and prioritizing long-term relationships with influencers.

## VII. Conclusion
Influencer marketing offers an invaluable approach to deepen customer relationships and drive growth. By emphasizing authenticity and adapting to the dynamic landscape, brands can leverage influencer partnerships effectively.

---

### Final Thoughts
Brands are encouraged to embrace these trends and adapt to ongoing changes to achieve excellence in their influencer marketing strategies. The emphasis on authenticity and measurable results will be essential for maintaining a competitive edge in 2025.

```
This markdown document is structured to promote clarity and readability for executive audiences. The information is presented in a logical flow with key trends, engagement strategies, challenges, and future directions outlined succinctly.

### Conclusion

In the agent task outputs above, you can see where the agent called the planning before starting the task. For more information on Planning, please reference our [documentation](https://docs.crewai.com/en/concepts/planning).

The Planing feature is an alternative to the **Reasoning** feature in CrewAI. There's a good chance both perform similar or the reasoning will be better.
We recommend trying both to see what works best for you.

Happy building! 🎉